# Промо для расширения покупаемого ассортимента в рамках отделов, которые еще не посещались

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import warnings

In [2]:
df = pd.read_csv('test_data.csv')

In [3]:
df

,Клиент,Отдел,Количество,Сумма,Товар,Время_суток,Отдел_номер
0,456851.0,ОТД БАКАЛЕЯ/КОНСЕРВЫ,1.000,57.9900,"КОРОТКОРЕЗ(РОЖ,УЛИТ)",Утро/День,4
1,456851.0,ОТД БАКАЛЕЯ/КОНСЕРВЫ,1.000,57.9900,"ДЛИННОРЕЗ(СПАГ,МАК)",Утро/День,4
2,500944.0,ОТД КОНДИТЕРСКИЕ ИЗДЕ,0.387,325.0760,КОНФ ШОКОЛАДНЫЕ ВЕС,Утро/День,7
3,500944.0,ОТД КОНДИТЕРСКИЕ ИЗДЕ,0.302,233.1410,КОНФ ШОКОЛАДНЫЕ ВЕС,Утро/День,7
4,331038.0,ОТД МЯСНОЙ ЦЕХ,0.154,70.8385,BAР КОЛБ-НAРЕЗКA(SB),Утро/День,12
...,...,...,...,...,...,...,...
3961156,2190129.0,ОТД АКЦИЯ ФИШКА NF2,1.000,3799.0000,AКЦИЯ ЛОЯЛ ТЕКСТИЛЬ,Утро/День,2
3961157,347020.0,ОТД АКЦИЯ ФИШКА NF2,1.000,3799.0000,AКЦИЯ ЛОЯЛ ТЕКСТИЛЬ,Утро/День,2
3961158,567863.0,ОТД АКЦИЯ ФИШКА NF2,1.000,3419.1000,AКЦИЯ ЛОЯЛ ТЕКСТИЛЬ,Утро/День,2
3961159,305152.0,ОТД АКЦИЯ ФИШКА NF2,1.000,3799.0000,AКЦИЯ ЛОЯЛ ТЕКСТИЛЬ,Утро/День,2


In [5]:
data = df[['Клиент', 'Отдел', 'Товар', 'Отдел_номер']].groupby(['Клиент', 'Отдел', 'Отдел_номер', 'Товар']).sum().reset_index()
data = data[data['Товар'] != 'ПАКЕТЫ']
data['Клиент'] = data['Клиент'].astype('int')
data

,Клиент,Отдел,Отдел_номер,Товар
0,2,ОТД ABТОПРИНAДЛЕЖНОСТИ,0,КЛЕИ
1,2,ОТД БАКАЛЕЯ/КОНСЕРВЫ,4,ГРЕЧКА ФАС
2,2,ОТД БАКАЛЕЯ/КОНСЕРВЫ,4,КСМ ЯПОНИЯ БАКАЛЕЯ
3,2,ОТД БАКАЛЕЯ/КОНСЕРВЫ,4,КУКУРУЗА
4,2,ОТД БАКАЛЕЯ/КОНСЕРВЫ,4,МАСЛИНЫ С\К
...,...,...,...,...
864850,3213513,ОТД ОВОЩИ/ФРУКТЫ,15,ПЕРСИКИ
864851,3213513,ОТД ОВОЩИ/ФРУКТЫ,15,СВЕКЛА
864852,3213579,ОТД БАКАЛЕЯ/КОНСЕРВЫ,4,ФАСОЛЬ КОНС
864853,3213579,ОТД МОЛОЧНЫЕ ПРОД.,10,СЫРКИ ГЛАЗИРОВАННЫЕ


# обычные рекомендации других популярных товаров

In [5]:
def suggest_promo_products(client_id):
    client_data = data[data['Клиент'] == client_id]
    department_counts = client_data.groupby('Отдел')['Товар'].value_counts()

    suggested_products = {}

    for department, product in department_counts.index:
        count = department_counts.loc[department, product]
        if department not in suggested_products or count > suggested_products[department]['Количество']:
            suggested_products[department] = {'Товар': product, 'Количество': count}

    return suggested_products

In [9]:
client_id = 10.0
suggested_products = suggest_promo_products(client_id)

for department, product_info in suggested_products.items():
    product = product_info['Товар']
    count = product_info['Количество']
    print(f"Отдел: {department}, Рекомендуемый товар: {product}")

Отдел: ОТД БАКАЛЕЯ/КОНСЕРВЫ, Рекомендуемый товар: ПАНИРОВКА
Отдел: ОТД КОСМЕТИКА/БЫТ.ХИМ�, Рекомендуемый товар: ПРОКЛАД Д/КРИТИЧ ДН
Отдел: ОТД МОЛОЧНЫЕ ПРОД., Рекомендуемый товар: ЯЙЦО ДЕРЕВ/ЗДОРОВЬЕ
Отдел: ОТД ОВОЩИ/ФРУКТЫ, Рекомендуемый товар: ДЫНИ
Отдел: ОТД ПТИЦА/ЗАМОРОЖЕННЫ, Рекомендуемый товар: МОРОЖЕНОЕ СТАКАНЧИК
Отдел: ОТД СЫРЫ/САЛАТЫ/ПОЛУФ�, Рекомендуемый товар: СЫР ТВ,П/ТВ ПЛЕН ВЕС
Отдел: ОТД ТЕКСТИЛЬ, Рекомендуемый товар: ЖЕН ТРУСЫ
Отдел: ОТД ХЛЕБ/ТОРТЫ, Рекомендуемый товар: ИЗДЕЛИЯ СДОБНЫЕ


# Рекомендации на основе K-means

In [6]:
# Шаг 1: Кластеризация методом k-средних
def perform_kmeans_clustering(data, num_clusters):
    # Подготовка данных для кластеризации
    # Преобразование строкового столбца "Товар" в числовой формат
    le = LabelEncoder()
    data['Товар_код'] = le.fit_transform(data['Товар'])
    X = data[['Клиент', 'Товар_код']].values

    # Выполнение кластеризации методом k-средних
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(X)

    # Получение меток кластеров для каждого клиента
    cluster_labels = kmeans.labels_
    
    return cluster_labels

# Шаг 2: Выбор клиента и получение его метки кластера
def get_customer_cluster_label(customer_id, cluster_labels, customer_ids):
    index = customer_ids.index(customer_id)
    return cluster_labels[index]

# Шаг 3: Анализ покупок внутри кластера и формирование рекомендаций
def generate_recommendations(cluster_label, data, num_recommendations, customer_id):
    cluster_data = data[data['ClusterLabel'] == cluster_label]
    
    # Фильтрация покупок клиента в исходном отделе
    customer_department = data[data['Клиент'] == customer_id]['Отдел'].values[0]
    cluster_data = cluster_data[cluster_data['Отдел'] != customer_department]
    
    popular_products = cluster_data['Товар'].value_counts().head(num_recommendations)
    recommendations = popular_products.index.tolist()
    
    return recommendations

In [7]:
with warnings.catch_warnings(record = True):
    warnings.simplefilter("always")
    # Выполнение кластеризации методом k-средних
    cluster_labels = perform_kmeans_clustering(data, num_clusters=5)

    # Добавление столбца с метками кластеров в исходный датафрейм 'data'
    data['ClusterLabel'] = cluster_labels

    # Выбор клиента и получение его метки кластера
    customer_id = 10.0
    customer_cluster_label = get_customer_cluster_label(customer_id, cluster_labels, data['Клиент','Отдел'].tolist())

    # Генерация рекомендаций для выбранного клиента
    recommendations = generate_recommendations(customer_cluster_label, data, num_recommendations=10, customer_id=customer_id)

    # Вывод рекомендаций
    print(f"Рекомендации для клиента {customer_id}: {recommendations}")

KeyError: ('Клиент', 'Отдел')

В коде используется следующий принцип для формирования рекомендаций:

1. Кластеризация методом k-средних производится на основе данных о клиентах и покупках.
2. Для выбранного клиента определяется его метка кластера.
3. Анализируются покупки внутри кластера, к которому принадлежит выбранный клиент.
4. Из покупок внутри кластера выбираются наиболее популярные товары.
5. Формируются рекомендации, предлагая клиенту эти популярные товары.

Таким образом, рекомендации строятся на основе покупок других клиентов, принадлежащих к тому же кластеру, что и выбранный клиент. Предлагаются товары, которые наиболее часто покупаются внутри этого кластера, что может помочь клиенту обнаружить популярные или интересные товары, основываясь на поведении других клиентов из того же кластера.

In [20]:
import pandas as pd
import warnings
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

def perform_kmeans_clustering(data, num_clusters):
    le = LabelEncoder()
    data['Товар_код'] = le.fit_transform(data['Товар'])
    X = data[['Клиент', 'Товар_код']].values

    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(X)

    cluster_labels = kmeans.labels_

    return cluster_labels

def get_customer_cluster_label(customer_id, cluster_labels, customer_ids):
    index = customer_ids.index(customer_id)
    return cluster_labels[index]

def generate_recommendations(cluster_label, data, num_recommendations, customer_id):
    cluster_data = data[data['ClusterLabel'] == cluster_label]

    customer_department = data[data['Клиент'] == customer_id]['Отдел'].values[0]
    cluster_data = cluster_data[cluster_data['Отдел'] != customer_department]

    popular_products = cluster_data['Товар'].value_counts().head(num_recommendations)
    recommendations = popular_products.index.tolist()

    return recommendations

# Выполнение кластеризации методом k-средних
cluster_labels = perform_kmeans_clustering(data, num_clusters=5)

# Добавление столбца с метками кластеров в исходный датафрейм 'data'
data['ClusterLabel'] = cluster_labels

# Выбор клиента и получение его метки кластера
customer_id = 10.0
customer_cluster_label = get_customer_cluster_label(customer_id, cluster_labels, data['Клиент'].tolist())

# Генерация рекомендаций для выбранного клиента
recommendations = generate_recommendations(customer_cluster_label, data, num_recommendations=10, customer_id=customer_id)

# Создание датафрейма с рекомендациями
recommendations_df = pd.DataFrame({'Товар': recommendations})

# Объединение рекомендаций с информацией об отделе
recommendations_df = recommendations_df.merge(data[['Товар', 'Отдел']], on='Товар', how='left')

# Вывод рекомендаций в формате датафрейма
print(f"Рекомендации для клиента {customer_id}:")
recommendations_df[['Товар', 'Отдел']] = recommendations_df[['Товар', 'Отдел']].groupby(['Товар', 'Отдел']).sum().reset_index()
recommendations_df = recommendations_df.dropna()
recommendations_df[['Товар', 'Отдел']]

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Рекомендации для клиента 10.0:


,Товар,Отдел
0,КОНФ ШОКОЛАДНЫЕ ВЕС,ОТД КОНДИТЕРСКИЕ ИЗДЕ
1,МАЙОНЕЗ,ОТД МОЛОЧНЫЕ ПРОД.
2,МИН ВОДА РОС ПЭТ Б/Г,ОТД НАПИТКИ
3,ПОСУДA ОДНОРAЗОBAЯ,ОТД ТОВАРЫ Д/ ДОМА
4,РАЗДЕЛКА,ОТД ПТИЦА/ЗАМОРОЖЕННЫ
5,СBИН/ЧAСТЬ КР КУС(T),ОТД МЯСНОЙ ЦЕХ
6,СВЕКЛА,ОТД ОВОЩИ/ФРУКТЫ
7,СЫР ПЛАВЛЕННЫЙ ШТ,ОТД СЫРЫ/САЛАТЫ/ПОЛУФ�
8,"СЫР ТВ,П/ТВ ПЛЕН ВЕС",ОТД СЫРЫ/САЛАТЫ/ПОЛУФ�
9,СЫРКИ ГЛАЗИРОВАННЫЕ,ОТД МОЛОЧНЫЕ ПРОД.
